In [2]:
import gym
from gym import wrappers
import numpy as np
import tensorflow as tf

In [2]:
ACTIONS = 2

### CartPole with Agent manually trained by player

In [3]:
# env = gym.make('CartPoleU-v0')
gym.envs.register(
    id='CartPoleUV-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    max_episode_steps=5000,      # MountainCar-v0 uses 200
)

In [4]:
env = gym.make('CartPoleUV-v0')
env.mode = 'human'

[2017-12-22 16:16:52,874] Making new env: CartPoleUV-v0


In [5]:
def get_action_from_tanh(value):
    return 1 if value[0][0]>value[0][1] else 0
    

In [6]:
def reverse_action(action):
    return 1 if action == 0 else 0

In [18]:
IS_HUMAN_MODE = False

In [8]:
hist_observations = []
hist_actions = [] 

In [25]:
#!/usr/bin/env python
from __future__ import print_function
import time
import random
import sys, gym


# Test yourself as a learning agent! Pass environment name as a command-line argument.
#
window_size = 2

action_obser_hist = []

hist_observations_epoch = []
hist_actions_epoch = []

env = gym.make('CartPoleUV-v0')

if not hasattr(env.action_space, 'n'):
    raise Exception('Keyboard agent only supports discrete action spaces')
ACTIONS = env.action_space.n
ROLLOUT_TIME = 1000
SKIP_CONTROL = 0    # Use previous control decision SKIP_CONTROL times, that's how you
                    # can test what skip is still usable.

human_agent_action = 0
human_wants_restart = False
human_sets_pause = False
waiting_for_human_action = True

def key_press(key, mod):
    global human_agent_action, human_wants_restart, human_sets_pause, waiting_for_human_action
    
    if key==0xff0d: human_wants_restart = True
    if key==32: human_sets_pause = not human_sets_pause
        
    a = int( key - ord('0') )
    
    if a < 0 or a >= ACTIONS: return
    
    human_agent_action = a
    waiting_for_human_action = False

def key_release(key, mod):
    global human_agent_action, waiting_for_human_action
    a = int( key - ord('0') )
    if a < 0 or a >= ACTIONS: return
#     if human_agent_action == a:
#         human_agent_action = 0
#     waiting_for_human_action = True

env.render()
env.unwrapped.viewer.window.on_key_press = key_press
env.unwrapped.viewer.window.on_key_release = key_release

def rollout(env):
    global human_agent_action, human_wants_restart, human_sets_pause, waiting_for_human_action, hist_actions, hist_observations, hist_observations_epoch, hist_actions_epoch, action_obser_hist, window_size
    
    human_wants_restart = False
    obser = env.reset()
    skip = 0
    for t in range(ROLLOUT_TIME):
        # Human teaching the NN        
        if IS_HUMAN_MODE:
            if not skip:
                #print("taking action {}".format(human_agent_action))
                a = human_agent_action
                skip = SKIP_CONTROL
            else:
                skip -= 1
            
            hist_actions_epoch.append(a)
            hist_observations_epoch.append(obser)
            a = reverse_action(a)
        
        # NN balansing POLE by herself
        else:
            action_obser_hist.append(np.array(obser).reshape([1, 4]))
            a = model.predict(np.array(action_obser_hist).reshape([1, -1, 4]))
            a = get_action_from_tanh(a)
            a = reverse_action(a)
            
            if len(action_obser_hist) == window_size:    
                del action_obser_hist[0]
#                 print('Predicted action: ', a)
                
#             else: 
#                 a = random.choice([1,0])
            
        obser, r, done, info = env.step(a)

        
        if IS_HUMAN_MODE and done:
            hist_actions.append(hist_actions_epoch[:-10])
            hist_observations.append(hist_observations_epoch[:-10])
            hist_actions_epoch = []
            hist_observations_epoch = []
        
        elif not IS_HUMAN_MODE and done:
            action_obser_hist = []
            
        env.render()
        time.sleep(0.1)

        if done: break
        if human_wants_restart: break
        
        while waiting_for_human_action and IS_HUMAN_MODE:
            env.render()
            time.sleep(0.5)

print("ACTIONS={}".format(ACTIONS))
print("Press keys 1 2 3 ... to take actions 1 2 3 ...")
print("No keys pressed is taking action 0")

while 1:
    rollout(env)

[2017-12-22 16:26:30,439] Making new env: CartPoleUV-v0


ACTIONS=2
Press keys 1 2 3 ... to take actions 1 2 3 ...
No keys pressed is taking action 0


KeyboardInterrupt: 

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SimpleRNN, Dropout
from keras.optimizers import Adam, SGD


def get_model():
    model = Sequential()
    model.add(Dense(4, input_shape=[4], activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(6, activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(2, activation='tanh', kernel_initializer='random_uniform'))
    return model

def get_reccurent_model():
    model = Sequential()
#     model.add(LSTM(16, input_shape=(4, 1), return_sequences=False))
#     model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='tanh', kernel_initializer='random_uniform'))
    return model

def get_reccurent_stateful_model():
    model = Sequential()
#     model.add(SimpleRNN(1, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    # Maybe more neurons needed     
    # Not stateful
    model.add(LSTM(4, input_dim=4, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(4))
    model.add(Dropout(0.1))
    # Stateful
#     model.add(LSTM(16, batch_input_shape=(1, 4, 1), stateful=True, return_sequences=True))
#     model.add(LSTM(16, stateful=True))
#     model.add(SimpleRNN(4, batch_input_shape=(1, 4, 1), stateful=True, ))
#     model.add(LSTM(16, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='tanh', kernel_initializer='random_uniform'))
    return model

Using TensorFlow backend.


In [10]:
# Create chunks from mini parts
window_size = 2

X_chunked = []
Y_chunked = []

for i, obs in enumerate(hist_observations):
    obs = np.array(obs)

    for j in range(obs.shape[0] - window_size):
        X_chunked.append(obs[j:j+window_size])
        Y_chunked.append(hist_actions[i][j+window_size-1])

In [11]:
X = np.array(X_chunked)
Y = np.array(Y_chunked)
print(X.shape, Y.shape)

((0,), (0,))


In [ ]:
Y = np.concatenate([Y.reshape(-1, 1),np.zeros(Y.shape).reshape(-1, 1)], axis=1)
Y[:, 1][Y[:,0] == 0] = 1 

In [12]:
model = get_reccurent_stateful_model()
model.compile(Adam(), 'binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(4, return_sequences=True, input_shape=(None, 4))`


In [13]:
test = np.load('./data/AgentLearning/train_data.npz')
X = test['x']
Y = test['y']

In [15]:
X.shape

(767, 2, 4)

In [24]:
# Stateless training
model.fit(X, Y, verbose=1, epochs=10000, shuffle=False)

Epoch 1/10000
767/767 [==============================] - 0s - loss: 0.3714 - acc: 0.8377     
Epoch 2/10000
767/767 [==============================] - 0s - loss: 0.3589 - acc: 0.8435     
Epoch 3/10000
767/767 [==============================] - 0s - loss: 0.3894 - acc: 0.8279     
Epoch 4/10000
767/767 [==============================] - 0s - loss: 0.3925 - acc: 0.8318     
Epoch 5/10000
767/767 [==============================] - 0s - loss: 0.3808 - acc: 0.8240     
Epoch 6/10000
767/767 [==============================] - 0s - loss: 0.3736 - acc: 0.8435     
Epoch 7/10000
767/767 [==============================] - 0s - loss: 0.3875 - acc: 0.8246     
Epoch 8/10000
767/767 [==============================] - 0s - loss: 0.3755 - acc: 0.8175     
Epoch 9/10000
767/767 [==============================] - 0s - loss: 0.3800 - acc: 0.8338     
Epoch 10/10000
767/767 [==============================] - 0s - loss: 0.3972 - acc: 0.8325     
Epoch 11/10000
767/767 [==============================] - 0

767/767 [==============================] - 0s - loss: 0.3622 - acc: 0.8572     
Epoch 87/10000
767/767 [==============================] - 0s - loss: 0.3727 - acc: 0.8553     
Epoch 88/10000
767/767 [==============================] - 0s - loss: 0.3704 - acc: 0.8585     
Epoch 89/10000
767/767 [==============================] - 0s - loss: 0.3573 - acc: 0.8605     
Epoch 90/10000
767/767 [==============================] - 0s - loss: 0.3242 - acc: 0.8501     
Epoch 91/10000
767/767 [==============================] - 0s - loss: 0.3421 - acc: 0.8501     
Epoch 92/10000
767/767 [==============================] - ETA: 0s - loss: 0.3539 - acc: 0.850 - 0s - loss: 0.3576 - acc: 0.8422     
Epoch 93/10000
767/767 [==============================] - 0s - loss: 0.3616 - acc: 0.8377     
Epoch 94/10000
767/767 [==============================] - 0s - loss: 0.3524 - acc: 0.8468     
Epoch 95/10000
767/767 [==============================] - 0s - loss: 0.3430 - acc: 0.8488     
Epoch 96/10000
767/767 [===

767/767 [==============================] - 0s - loss: 0.3464 - acc: 0.8527     
Epoch 172/10000
767/767 [==============================] - 0s - loss: 0.3405 - acc: 0.8488     
Epoch 173/10000
767/767 [==============================] - 0s - loss: 0.3684 - acc: 0.8488     
Epoch 174/10000
767/767 [==============================] - 0s - loss: 0.3337 - acc: 0.8579     
Epoch 175/10000
767/767 [==============================] - 0s - loss: 0.3310 - acc: 0.8462     
Epoch 176/10000
767/767 [==============================] - 0s - loss: 0.3468 - acc: 0.8481     
Epoch 177/10000
767/767 [==============================] - 0s - loss: 0.3480 - acc: 0.8527     
Epoch 178/10000
767/767 [==============================] - 0s - loss: 0.3263 - acc: 0.8488     
Epoch 179/10000
767/767 [==============================] - 0s - loss: 0.3347 - acc: 0.8585     
Epoch 180/10000
767/767 [==============================] - 0s - loss: 0.3650 - acc: 0.8455     
Epoch 181/10000
767/767 [==============================]

767/767 [==============================] - 0s - loss: 0.3116 - acc: 0.8533     
Epoch 257/10000
767/767 [==============================] - 0s - loss: 0.3468 - acc: 0.8481     
Epoch 258/10000
767/767 [==============================] - 0s - loss: 0.3649 - acc: 0.8592     
Epoch 259/10000
767/767 [==============================] - 0s - loss: 0.3302 - acc: 0.8657     
Epoch 260/10000
767/767 [==============================] - 0s - loss: 0.3216 - acc: 0.8546     
Epoch 261/10000
767/767 [==============================] - 0s - loss: 0.3665 - acc: 0.8462     
Epoch 262/10000
767/767 [==============================] - 0s - loss: 0.3089 - acc: 0.8559     
Epoch 263/10000
767/767 [==============================] - 0s - loss: 0.3336 - acc: 0.8638     
Epoch 264/10000
767/767 [==============================] - 0s - loss: 0.3422 - acc: 0.8572     
Epoch 265/10000
767/767 [==============================] - 0s - loss: 0.3358 - acc: 0.8501     
Epoch 266/10000
767/767 [==============================]

KeyboardInterrupt: 

In [ ]:
# ! not tested method
# Statefull training
for i in range(100000):
    if not i % 10:
        flag_verb = 1
        print('Epoch: ', i)
    else:
        flag_verb = 1
        
    model.fit(X_train.reshape((-1, 4, 1)), y_train, epochs=1, validation_data=[X_test.reshape((-1, 4, 1)), y_test], batch_size=1, verbose=flag_verb, shuffle=False)
    model.reset_states()

In [203]:
np.savez_compressed('./data/AgentLearning/train_data', x=X, y=Y)